In [ ]:
from importlib import reload
import scraper
## save models, data...
import stanza
reload(scraper)

In [ ]:
nlp_stanza = stanza.Pipeline(lang='fr',processors='tokenize,mwt,lemma,pos,depparse')

In [ ]:
def corpus_builder(label,text):
    # run stanza
    stanza_obj = nlp_stanza(text)
    final_sentences = []
    temp_sents = stanza_obj.sentences
    with open("corpus.xml","a+") as f:
        f.write("<art label=\""+label+"\">\n")
        for sent in temp_sents:
            f.write("<s>\n")
            temp_words = sent.words
            # write word
            for word in temp_words:
                # if word.lemma.lower() not in myStopwords:
                f.write(str(word.id)+"\t"+word.text+"\t"+word.lemma+"\t"+word.upos+"\t"+str(word.head)+"\t"+word.deprel+"\n") 
            # end sentence tag
            f.write("</s>\n")
        f.write("</art>\n")

In [ ]:
df_complete = scraper.decompress_pickle("corpus_complet_5themes.pbz2")

In [ ]:
# get a sample of 9200 articles for eco politique et societe
df_sampled = df_complete[df_complete.label.isin(['politique','societe','economie'])]
df_sampled = df_sampled.groupby('label').head(9200) # 27600 au total

In [ ]:
import re
lemonde = open('corpus.xml', encoding='utf-8').read()
# split articles
lemonde_articles = re.split('</art>', lemonde)

In [ ]:
## Build dataframe lists
# all the articles
articles_output = []
# all the labels
labels_output = []
for article in lemonde_articles:
    sentences = ""
    # extract the label, exclude blank lines
    if re.split(r"</?s>",article)[0] != "\n":
        label = re.split(r"</?s>",article)[0].split("\"")[1]
        labels_output.append(label)
    for tokens in re.split(r"</?s>",article)[1:]:
        if tokens != "\n":
            for token in tokens.split("\n"):
                if token:
                    # remove stop words
                    if token.split("\t")[3].startswith("PROPN") or token.split("\t")[3] in ["NOUN","VERB","ADV","ADJ","SYM","INTJ"]:
                        sentences += token.split("\t")[2] + " "
        sentences += "\n"
    if sentences:
        articles_output.append(sentences)

In [ ]:
df_total = scraper.build_df(labels_output,articles_output)

In [ ]:
df_1000 = df_total.groupby('label').head(1000) # 3000 au total

In [ ]:
df_8200 = df_total.groupby('label').tail(8200) # 24600 au total

In [ ]:
with open("final1000.pickle","wb") as p:
    pickle.dump(df_1000,p)

In [ ]:
with open("final8200.pickle","wb") as p:
    pickle.dump(df_8200,p)